<a href="https://colab.research.google.com/github/kssaril/machine-learning/blob/master/Predict_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Can a neural network  predict what Database to be used for my application Design ?**

In [0]:

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy
import pandas
import io
import requests
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


**Training an Artificial Neural Network**
* Training an artificial neural network involves the following steps:
* Weights are randomly initialized to numbers that are near zero but not zero.
* Feed the observations of your dataset to the input layer.
* Forward propagation (from left to right): neurons are activated and the predicted values are obtained.
* Compare predicted results to actual values and measure the error.
* Backward propagation (from right to left): weights are adjusted.
* Repeat steps 1–5
One epoch is achieved when the whole training set has gone through the neural network.

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

Now load the dataset to train the model . The dataset formatted as ***TPS,data Volume,CucurrentUsers,TargetRespTime,DB***

In [0]:
# load dataset
import pandas as pd
import io
import requests
url="https://raw.githubusercontent.com/kssaril/machine-learning/master/databasepropsdat.csv"
content=requests.get(url).content
dataframe=pd.read_csv(io.StringIO(content.decode('utf-8')),header=None)

#dataframe = pandas.read_csv("/Users/kamas012/Proj/ML/Learn/databasepropsdat.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

Test the data 

In [0]:
X[1]

array([63. , 33. , 47. ,  1.6])

In [0]:
Y[:5]

array(['Oracle', 'DynamoDB', 'Oracle', 'Oracle', 'DynamoDB'], dtype=object)

We need to hot encode the Labels

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
hot_encoded_y = np_utils.to_categorical(encoded_Y)

Sample hot encoded label

In [0]:
hot_encoded_y[:10]

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

**Define the Neural Network**
* There is no rule of thumb as to how many nodes you should add. However, a common strategy is to choose the number of nodes as the average of nodes in the input layer and the number of nodes in the output layer.

* First param - number of nodes you want to add to this layer 
* The Rectified Linear Unit is the most commonly used activation function in deep learning models. 
* Compiling the ANN need optimizer which helps in getting optimized set of weights in the NN 
metrics - Criteria used to evaluate the model

In [0]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

Create classifier using KerasClassifier to train the model using the data with 200 epochs 

In [0]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

**evaluate the model**
<br>
In k-fold cross-validation, the original sample is randomly partitioned into k equal sized subsamples. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k − 1 subsamples are used as training data

In [0]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)


**Evaluate the model and print results**



In [0]:
results = cross_val_score(estimator, X, hot_encoded_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Baseline: 68.79% (35.29%)


Split the Data into train sets and test set  and 
train the model with train data

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, hot_encoded_y, test_size=0.33, random_state=seed)
estimator.fit(X_train, Y_train)

Ask the model to predict what DB  the best choice for a set of input data arrays . Here I use the test data set  ?

In [0]:
predictions = estimator.predict(X_test)
print(encoder.inverse_transform(predictions))

['DynamoDB' 'Oracle' 'DynamoDB' 'Oracle' 'Oracle' 'DynamoDB' 'DynamoDB'
 'MariaDB' 'DynamoDB' 'MariaDB' 'DynamoDB' 'DynamoDB' 'Oracle' 'Oracle'
 'Oracle' 'MariaDB' 'MariaDB' 'MariaDB' 'Oracle' 'DynamoDB' 'Oracle'
 'DynamoDB' 'Oracle' 'DynamoDB' 'Oracle' 'DynamoDB' 'DynamoDB' 'MariaDB'
 'DynamoDB' 'Oracle' 'Oracle' 'Oracle' 'Oracle' 'MariaDB' 'DynamoDB'
 'MariaDB' 'MariaDB' 'DynamoDB' 'DynamoDB' 'Oracle' 'Oracle' 'DynamoDB'
 'DynamoDB' 'Oracle' 'DynamoDB' 'MariaDB' 'MariaDB' 'Oracle' 'DynamoDB'
 'DynamoDB' 'DynamoDB' 'MariaDB']


**Lets try what the model will predict for a single input **

In [0]:
from numpy import array
data = array([[90. , 35. , 10. ,  1.2]])
predictions = estimator.predict(data)
print(encoder.inverse_transform(predictions))

['MariaDB']
